In [1]:
import os
import sys
import json
import numpy as np
import skimage.draw

In [2]:
# Root directory of the project
ROOT_DIR = r"D:\Final Year Project\0000_CODE"
sys.path.append(ROOT_DIR)

# Import Mask RCNN
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

U:\ANACONDA\envs\mask_env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
U:\ANACONDA\envs\mask_env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
U:\ANACONDA\envs\mask_env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
U:\ANACONDA\envs\mask_env\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

In [3]:
############################################################
#  Configuration for training on our custom dataset.
#  Derives from the base Config class and overrides some values.
############################################################
class CustomConfig(Config):
    # Configuration name
    NAME = "object"

    GPU_COUNT = 2
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 6  # Background + 6 classes

    STEPS_PER_EPOCH = 200
    VALIDATION_STEPS = 10
    
    # Skip detections with < 80% confidence
    DETECTION_MIN_CONFIDENCE = 0.8

In [4]:
#####################################################################################
#  Customising the Dataset class in Mask R-CNN utilities with our Custom Dataset
#####################################################################################

class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """
        Loading a subset of our Dataset (Train/Val)

        dataset_dir: Root directory of the dataset.
        subset:      Subset to load: train or val
        """
        # Adding the 6 classes.
        self.add_class("object", 1, "person")
        self.add_class("object", 2, "rider")
        self.add_class("object", 3, "car")
        self.add_class("object", 4, "bus")
        self.add_class("object", 5, "truck")
        self.add_class("object", 6, "electric auto")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        annotations1 = json.load(open(r"D:\Final Year Project\0000_CODE\Dataset\train\final_train.json"))
        annotations = list(annotations1.values())

        if(subset == "val"):
            annotations1 = json.load(open(r"D:\Final Year Project\0000_CODE\Dataset\val\val-json.json"))
            annotations = list(annotations1.values())

        # Skipping un-annotated images
        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
            
            # Getting the object boundary coordinates from the annotation json file
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['names'] for s in a['regions']]
            print("objects:",objects)

            name_dict = {"person": 1,"rider": 2,"car": 3,"bus": 4,"truck": 5,"electric auto": 6}

            num_ids = [name_dict[a] for a in objects]
            print("numids",num_ids)

            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)

            # load_mask() needs the image size to convert polygons to masks.
            height, width = image.shape[:2]

            #Adding the image
            self.add_image(
                "object",
                image_id=a['filename'],
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """
        Generate instance masks for an image.

       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """

        # Converting the polygons to a bitmap mask of shape [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Getting indexes of pixels inside the polygon and setting them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
        	mask[rr, cc, i] = 1

         
        # Mapping class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)

        # Returning the mask, and array of class IDs of each instance.
        return mask, num_ids

    def image_reference(self, image_id):
        # Returns the path of the image.
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [5]:
def train(model):
    # Preparing the train & val datasets.
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom(r"D:\Final Year Project\0000_CODE\Dataset", "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom(r"D:\Final Year Project\0000_CODE\Dataset", "val")
    dataset_val.prepare()

    # Training the network heads of a pre-trained model
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=11,
                layers='heads')

In [6]:
#Defining the model in training mode
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


In [7]:
#Defining the MS-COCO pre-trained model weights path.
weights_path = COCO_WEIGHTS_PATH
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

In [8]:
checkpoint_path = r"D:\Final Year Project\0000_CODE\logs\object20220421T1328\mask_rcnn_object_0010.h5"
weights_path = checkpoint_path

In [9]:
#Loading MS-COCO pre-trained weights onto the defined model.
model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

Re-starting from epoch 10


In [11]:
#Train the defined model.
train(model)

objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person', 'person']
numids [1, 1]
objects: ['person', 'person']
numids [1, 1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person', 'person']
numids [1, 1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
objects: ['person']
numids [1]
